In [ ]:
import pandas as pd
import re
import unicodedata

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving en.openbeautyfacts.org.products.csv to en.openbeautyfacts.org.products.csv


In [ ]:
df = pd.read_csv('en.openbeautyfacts.org.products.csv', sep="\t",encoding="utf-8")

/tmp/ipython-input-3807133151.py:1: DtypeWarning: Columns (0,11,34,36,54,60,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('en.openbeautyfacts.org.products.csv', sep="\t",encoding="utf-8")


In [ ]:
df.columns


Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime',
       ...
       'water-hardness_100g', 'choline_100g', 'phylloquinone_100g',
       'beta-glucan_100g', 'inositol_100g', 'carnitine_100g', 'sulphate_100g',
       'nitrate_100g', 'acidity_100g', 'carbohydrates-total_100g'],
      dtype='object', length=204)

In [ ]:
df_brand = df[["code","product_name","brands"]].copy()

In [ ]:
df_brand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60957 entries, 0 to 60956
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   code          60957 non-null  object
 1   product_name  38731 non-null  object
 2   brands        33063 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [ ]:
nuovo_nome = 'brand_name'
df_brand = df_brand.rename(columns={'brands': nuovo_nome})


In [ ]:
df_brand.isna().sum()

,0
code,0
product_name,22226
brand_name,27894


In [ ]:
df_brand.shape

(60957, 3)

3. **Normalizzazione nomi dei brand**

Gli obiettivi principali:
- rimuovere brand senza nome

- rimuovere spazi extra, maiuscole/minuscole

- uniformare accenti, apostrofi, trattini

- gestire casi di brand multipli presenti nel nome

In [ ]:

def pulizia_normalizzazione(text):
    if pd.isna(text):
        return None

    text = str(text).strip().lower()

    if text in {'null', 'nan', '','unknown','no name', 'sans marque'}:
        return None

    # normalizzazione unicode e rimozione dei diacritici
    text = unicodedata.normalize('NFD', text) #NFD scompone le lettere separando gli accenti
    text = ''.join(
        char for char in text
        if unicodedata.category(char) != 'Mn' #Mn sono gli accenti --> Il codice mantiene le lettere non accentate
    )

    # minuscolo
    text = text.lower()

    # uniforma separatori
    text = re.sub(r"[;&/|]", ",", text)
    text = re.sub(r"\band\b", ",", text)

    # pulizia caratteri
    text = re.sub(r"[^a-z0-9, ]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    # split brand
    brands = [b.strip() for b in text.split(",") if b.strip()]

    # deduplicazione mantenendo l'ordine
    brands = list(dict.fromkeys(brands))

    # primo brand dominante
    return brands[0] if brands else None




In [ ]:
#applicazione del codice a tutto il dataset
df_brand['brand_name'] = df_brand['brand_name'].apply(pulizia_normalizzazione)

In [ ]:
#Rimozione dei brand con valore nullo
df_brand = df_brand.dropna(subset=['brand_name'])


Verifica l'esistenza nel dataset di brand con nome sconosciuto

In [ ]:
df_brand.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32881 entries, 1 to 60955
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   code          32881 non-null  object
 1   product_name  31056 non-null  object
 2   brand_name    32881 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [ ]:
(df_brand["brand_name"] == "unknown").sum()


np.int64(0)

Verifica per sapere quante righe sono state eliminate e se sono rimasti valori nulli

In [ ]:
df_brand.shape

(32881, 3)

In [ ]:
df_brand.isna().sum()

,0
code,0
product_name,1825
brand_name,0


Normalizzazione brand multipli:

In presenza di stringhe contenenti più denominazioni di brand, è stato mantenuto esclusivamente il primo elemento, interpretato come brand dominante del prodotto. Le occorrenze successive sono state considerate rumore descrittivo (divisioni aziendali o sotto-marchi) e non veri casi di co-branding.

Creazione brand_id : a ogni brand viene assegnato un identificativo univoco

In [ ]:
df_brand['brand_id'] = df_brand['brand_name'].astype('category').cat.codes


Creazione brand_segment :
- 3 categorie : luxury, middle, mass_market

In [ ]:
LUXURY = ['sea hair', 'apple', 'la savonnerie', 'kerastase', 'wild', 'nu skin', 'beauty of japan','finesse',
                 'estee lauder','beesmade','deneva skincare','equate','equate beauty','elemis','milani',
                 'aztec secret', 'herbalife', 'reviive', 'ariix', 'guerlain paris','	squid fur ology',
                 'kilian','mlle agathe','the body bloom','sephora','pink 21','pink21', 'kenzo', 'amway',
                  'awapul','aveda','too faced','too faced cosmetics','avalon','avalon cosmetics','paula s choice',
                  'dr brandt','dermalogica','jean mi de paris','yon ka paris','juice beauty','mineral fusion',
                 'olaplex','redken all soft','peter thomas roth','peterthomasroth','shiseido','anastasia beverly hills',
                'ahava','evanhealy', 'matrix','tom ford','burberry','payot','cartier','lancome','clavin klein','patyka',
                  'elisabeth arden','first aid beauty','fresh','galinee','galenic',
                  'gemology','guinot','hanskin','huda beauty','kiehl s','lancome',
                  'lancome paris','laneige','molton brown','murad','niod','philosophy',
                  'qiriness','rituals','sol de janeiro','tula','typology',
                 'origins','gucci','hugo boss','wholly kaw','redken','tatcha','murad','mac','m a c','vichy',
                 'mac make up art cosmetics','mac cosmetics','leclerc','givenchy','filorga','chanel','sooa',
                 'hermes','hermes paris','guerlain','guerlain l institut','dior','christian dior','collistar','wella professionals',
                 'yves saint laurent','jean paul gaultier', 'versace','loewe','herrera', 'Carolina Herrera','moroccanoil','deborah milano','bionike',
                 'isdin','ringana','gatineau','korres','apivita','aprivita','lumene','natura','fanola','davines','dermalogica',
                  'aesop','amika','bareminerals','benefit','belif','brickell','briogeo','charlotte tilbury','libre yves saint laurent']
MIDDLE = ['yves rocher', 'trick or treat', 'yardley', 'oat haus','fragonard' ,'aroma zone','acure',
                   'aliver', 'native', 'embryolisse','body earth inc', 'paul mitchell','dkny stories','mugler',
                   'booda','kedsal','kerasoin','victoria s secret','bath','american crew','hempz','pacifica',
                   'maui', 'maui moisture', 'lili of the desert','gold bond','marc anthony','naka platinum',
                   'truskin','rabanne','paco rabanne','biotherm','douce nature','montblanc','decleor','chanvria',
                   'natracouticals q defend','saryane sas','ekolep','	hada labo tokyo','mylee','cosmia','palladio',
                   'la roche posay','weleda','lavera','belle et bio','harmonie verte','massato','kap','desert essence',
                   'caprina','doo gro','nutiva','rinju','petal fresh','petal fresh organics','petal fresh superfoods',
                   'yes to','yesto','dr teal s','carol s daughter','earth mama organics','wella','eugene perma','nuxe paris',
                   'clinerience','akane','ylae','blancreme paris','cerave','mixa','aphogee','clinique','clinique for men',
                   'eucerin','monsavon','klorane','mustela','palmer s','bayliss','jergens','caudalie','japonesque','mrs meyer s',
                   'alba botanica','mario badescu','bioderma','mustela','ultrasun','shea moisture','olivella','horace',
                   'dr squatch','cetaphil','neem aura','uriage','nuxe','dermaclay','dr hauschka','sanoflore','sisley',
                   'narciso rodriguez','biolage','lea nature','peggy sage','centifolia','cosmia bio','arganicare',
                   'phyto paris','aplhanova','nuxe sun','rituals','uriage','ziaja','john frieda','biore','the body shop','green people',
                   'bulldog','happy naturals','upcircle','woowoo','grum','waken','hawaiian tropic',
                    'biosolis','o keeffe s',"o'keeffe's",'urtekram','athena','nacomi','eveline',
                    'love beauty','geek','bio oil','doterra','cadum','bioten','omia','babaria',
                  'eudermin','revlon','vidal','biorepair','heno de pravia','flor de mayo',
                  'les cosmetiques','agrado','etnia','sanex','365 whole foods','a derma','a vogel','absolution','acm','acorelle',
                  'activilong','acure','aderma','advanced clinicals','agarta','akileine',
                  'akane','alphanova','alpecin','alvadiem','alverde','alterra','clemence','cocon','coconut care','comme avant','cooper',
                  'corine de farme','coslys','cosmo naturel','creme of nature',
                  'dabur','cussons','dawn','dax','delhaize','define','demaso','demak up','deroche',
                  'derma','dermica','dermomed','dettol','dmp','doctissimo','les cosmetiques design paris',
                  'dodot','doliprane','dop','douglas','dr greve', 'larocheposay',
                  'alterra naturkosmetik','aveeno','avene','avon','avril','azarine',
                  'badger','ballot flurin','barnangen','batiste','beauterra','benecos',
                  'bioderma','biogran','biolage','biolane','biomed','biopha','biotique',
                  'biovie','body nature','boiron','botanics','bourjois','bubchen',
                  'burt s bees','byphasse','cadum','cattier','centifolia',
                  'christian lenart','cicabiafine','apaisyl','apoteket','aprolis',
                  'aquaphor','arbre vert','argiletz','art naturals','artdeco','assanis',
                  'attitude','auriege','earth mama organics','eau thermale avene','eco lips','ecover',
                  'edel white','elgydium','emma noel','endro','energie fruit',
                  'eo','equilibra','evoluderm','expanscience','faith in nature',
                  'farline','fenjal','fleurance nature','florame','florena',
                  'foamie','forces de nature','freeman','frosch','furterer',
                  'galderma','galenco','garancia','gehwol','genove','georganics',
                  'gerovital','gifrer','glow lab','gold bond','goodmark',
                  'graham hill','granado','gravier','greendoor','guhl','hairwell',
                  'happybrush','harry s','hask','hei poa','hey humans','himalaya',
                  'himalaya botanique','hipp','hismile','ho karan','honest',
                  'horace','hydrophill','i provenzali','idc institute','idraet',
                  'inava','indulekha','innisfree','instituto espanol','isana',
                  'isdin','jardin d oleane','jason','jeanne en provence',
                  'joico','jonzac','jowae','juice beauty','juvamine','k pour karite',
                  'kaliya beauty','karethic','keranove','keune','kevin murphy',
                  'khadi','kirk s natural','kiss my face','klorane','kneipp',
                  'la corvette','la provencale','la rosee','la saponaria',
                  'lamazuna','lavera','le comptoir du bain','lea nature',
                  'lea nature so bio etic','luxeol','lush','mademoiselle bio',
                  'mamaearth','manava','marilou bio','marius fabre','marvis',
                  'melvita','merci handy','meridol','mixa','moogoo','nacomi',
                  'naif','natessance','natracare','natura ekos','natura siberica',
                  'naturado','neal s yard','neobulle','neutraderm','neutrogena',
                  'noreva','novexpert','now solutions','nyx','ogx','oleant',
                  'omum','one with nature','ordinary','orientana','origins',
                  'pachamamai','panier de sens','patanjali','patrice mulato',
                  'petrole hahn','phbio','phyto','polaar','pranarom',
                  'pulpe de vie','puressentiel','radical','rainett',
                  'rampal latour','rausch','raw sugar','regenerate','rene furterer',
                  'rilastil','ringana','roc','roge cavailles','rubelia',
                  'ruby rose','saforelle','sanoflore','sante','saryane',
                  'schaebens','seba med','sensilis','sesderma','shea moisture',
                  'signal','simple','so bio','so bio etic','speick','st ives',
                  'sukin','svr','tea tree','terra naturi','tetesept','thalgo',
                  'thayers','the humble co','therabreath','timotei','tiki tahiti',
                  'topicrem','treaclemoon','urban hydration','urtekram',
                  'ushuaia','vitry','vlcc','waken','wholly kaw','wild',
                  'yardley london','yesto','zao','ziaja']

def segmento_brand(brand):
    brand = brand.lower().strip()
    if brand in LUXURY:
        return 'luxury'
    elif brand in MIDDLE:
        return 'middle'
    elif brand == 'unknown':
        return 'unknown'
    else:
        return 'mass_market'


In [ ]:
df_brand['brand_segment'] = df_brand['brand_name'].apply(segmento_brand)


In [ ]:
df_brand = df_brand.sort_values('brand_name')

In [ ]:
df_brand

,code,product_name,brand_name,brand_id,brand_segment
1,14,Süßlupinen Mehl,bodylab 24,1143,mass_market
3,92,airpods,apple,425,luxury
6,311,Huile végétale Amande douce,aroma zone,497,middle
7,315,Colgate Sensitive Plus. Made in India.,oat haus,5475,middle
8,320,"Body & Earth, Inc",body,1126,mass_market
...,...,...,...,...,...
60947,99420060,Botanica,hakawerk,3280,mass_market
60951,9978135848778,Pigmento,pink 21,5928,luxury
60952,99926632,Gommage lubrifiant poudre de Baikal Purifiante,yves rocher,7826,middle
60953,99941208,Zincodermil shampoo,istituto elvetico sanders,3683,mass_market


In [ ]:
df_brand['brand_segment'].value_counts()


,count
brand_segment,
mass_market,24989
middle,6980
luxury,912


Esplorazione brand per vedere quali sono i brand più popolari nel dataset

In [ ]:
df_brand['brand_name'].value_counts().head(50)

,count
brand_name,
unilever,1331
l oreal,1237
nivea,975
deliplus,527
henkel,511
carrefour,511
yves rocher,480
l oreal norge as,462
cosmia,376


Pulizia e normalizzazione di "product_name"
- Rimozione prodotti senza nome
- Sostituzione apostrofi e trattini con spazio
- Rimozione caratteri speciali
- Rimozione spazi


In [ ]:
def pulizia_normalizzazione_product(text):
    # gestione valori mancanti e casi ambigui
    if pd.isnull(text):
        return None

    text = str(text).strip().lower()

    if text in {'', 'null', 'nan', 'no name'}:
        return None

    # normalizzazione unicode e rimozione accenti
    text = unicodedata.normalize('NFD', text)
    text = ''.join(
        c for c in text
        if unicodedata.category(c) != 'Mn'
    )

    # sostituisce separatori comuni con spazio
    text = re.sub(r"[-_/']", " ", text)

    # rimuove caratteri non alfanumerici
    text = re.sub(r"[^a-z0-9 ]", " ", text)

    # normalizza spazi multipli
    text = re.sub(r"\s+", " ", text).strip()

    return text



In [ ]:
df_brand['product_name'] = df_brand['product_name'].apply(pulizia_normalizzazione_product)


In [ ]:
df_brand = df_brand.dropna(subset = ['product_name'])

In [ ]:
df_brand.head(60)

,code,product_name,brand_name,brand_id,brand_segment
2879,88582000939,2,1,0,mass_market
4772,0650240025396,zan zusi b b flash 30,1,0,mass_market
36568,5013965698897,1,1,0,mass_market
17455,3560070084074,deo men marine,1 de carrefour,1,mass_market
41807,6134598000044,savon liquide main life,100 da,2,mass_market
40271,5943044010046,gerocossen,12,3,mass_market
9051,20546472,cien,12,3,mass_market
25404,3700216252688,glycerine vegetale,123gelules,4,mass_market
58046,8859423200212,habino,13000,5,mass_market
41548,6130460000747,,150da,6,mass_market


Verifica finale:
- forma del dataset (quante istanze sono rimaste) -> 31062
- quanti valori nulli sono rimasti
- prodotti con nome sconosciuto

In [ ]:
df_brand.shape

(31056, 5)

In [ ]:
df_brand.isna().sum()

,0
code,0
product_name,0
brand_name,0
brand_id,0
brand_segment,0


In [ ]:
(df_brand['product_name'] == 'unknown').value_counts()

,count
product_name,
False,31056


In [ ]:
(df_brand['brand_segment'] == 'mass_market').value_counts()

,count
brand_segment,
True,23544
False,7512


In [ ]:
df_brand.tail(50)

,code,product_name,brand_name,brand_id,brand_segment
39892,5901887046639,gdanskin,ziaja,7863,middle
39853,5901887008460,hand cream,ziaja,7863,middle
39894,5901887049241,face cream,ziaja,7863,middle
39895,5901887050377,ziaja,ziaja,7863,middle
39896,5901887053507,natural care shower gel,ziaja,7863,middle
39891,5901887045243,jeju m oda skora,ziaja,7863,middle
39880,5901887035442,goat s milk hair conditioner with keratin,ziaja,7863,middle
39878,5901887034919,argan oil face cream,ziaja,7863,middle
39867,5901887027829,acondicionador oliva,ziaja,7863,middle
39855,5901887010371,eye make up remover,ziaja,7863,middle


In [ ]:
df_brand.to_excel("df_brand.xlsx", index= False)


**Importazione dataset con aggiunta della categoria di riferimento per**
- Creazione affinity matrix : cioè per ogni coppia di brand calcoliamo quanto sono “vicini” o “simili” dal punto di vista del mercato (segmento)

